## Assignment 9 - Monica Zhang
https://spark.apache.org/docs/latest/ml-features#lsh-operations  
https://databricks.com/blog/2017/05/09/detecting-abuse-scale-locality-sensitive-hashing-uber-engineering.html

In [1]:
import sys
print(sys.version)
print(spark.version)

3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:46:39) 
[GCC 10.4.0]
3.1.3


In [2]:
import os
import subprocess
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')
from itertools import compress 
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings(action='ignore')

#### Add "eagerEval.enabled" to beautify the way Spark DF is displayed

In [4]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [5]:
df = spark.read.parquet('gs://msca-bdp-students-bucket/shared_data/monicazhang/final_project_eda_154M_2.parquet')

23/03/09 18:08:01 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
df.limit(5)

coordinates,created_at,favorite_count,favorited,filter_level,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,lang,place,quote_count,quoted_status,quoted_status_id,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,entities,extended_entities,extended_tweet,geo,place.country,place.full_name,place.name,place.place_type,user.id,user.name,user.screen_name,user.location,user.created_at,user.description,user.followers_count,user.friends_count,user.listed_count,user.favourites_count,user.statuses_count,user.url,user.verified,retweets_count,user_type
null,Mon Jul 04 21:47:...,0,false,low,1544075457250955266,brad_px,1544075454868594688,1411214388544016384,false,en,null,0,null,null,null,0,0,,null,null,"<a href=""http://t...",little kids who h...,1656971251200,true,little kids who h...,"{false, Sat Jul 0...","{[], null, [], [{...",null,"{[0, 278], {[], n...",null,null,null,null,null,1411214388544016384,Px Brad,brad_px,null,Sat Jul 03 06:51:...,PC Guy hanging ou...,34,160,0,2198,3048,null,false,null,other
null,Mon Jul 04 21:47:...,0,false,low,1544075543322038274,MKLiebmann,1544074400160329728,496255842,false,en,null,0,null,null,null,0,0,,null,null,"<a href=""http://t...",We all want thing...,1656971271721,true,We all want thing...,"{false, Sat Feb 1...","{[], null, [], [{...",null,"{[0, 241], {[], n...",null,null,null,null,null,496255842,MK,MKLiebmann,"West Hollywood, CA",Sat Feb 18 19:30:...,Multidisciplinary...,812,282,45,5742,266,null,false,null,other
null,Mon Jul 04 21:48:...,0,false,low,1544075654760734723,FreeBeacon,1543989283115130887,455764741,false,en,null,0,null,null,null,0,0,,SaysSimonson Sinc...,null,"<a href=""https://...",@FreeBeacon @Says...,1656971298290,true,@FreeBeacon @Says...,"{false, Tue Feb 1...","{[], null, [], [{...",null,"{[26, 189], {[], ...",null,null,null,null,null,20487617,Choral Reave,ChoralReave,null,Tue Feb 10 02:50:...,The future is pro...,359,521,6,63630,72256,null,false,null,other
null,Mon Jul 04 21:48:...,0,false,low,1544075729800990720,null,null,null,false,en,null,0,null,null,null,0,0,RT,RWPUSA,"{null, Sun Jul 03...","<a href=""http://t...",RT @RWPUSA: Inste...,1656971316181,false,Instead of just f...,"{false, Fri Mar 0...","{[], null, [], []...",null,null,null,null,null,null,null,1499819718051614731,East Coast Girl ☮️,Lorelei_6886,"Waterford, CT, U.S.",Fri Mar 04 18:51:...,🎶🎭📚🎫🏀 Gen-X ...,346,1282,0,4709,1035,null,false,34,other
null,Mon Jul 04 21:48:...,0,false,low,1544075766551695360,null,null,null,false,en,null,0,null,null,null,0,0,RT,kim_siever,"{null, Mon Jul 04...","<a href=""http://t...",RT @kim_siever: R...,1656971324943,false,Remember when we ...,"{false, Mon Aug 2...","{[], null, [], []...",null,null,null,null,null,null,null,767850928183521280,Marlenus of Ar,MarlenusAr,null,Mon Aug 22 22:28:...,null,40,274,0,6526,2752,null,false,34,other


In [7]:
df = df.withColumn('hashtags', expr("entities.hashtags.text"))

In [8]:
df.count()

1541655

In [9]:
df_location_counts = df.groupBy("user.location").count().orderBy(desc("count"))
df_location_counts.show(20, truncate=False)

+---------------+------+
|location       |count |
+---------------+------+
|null           |585096|
|United States  |28490 |
|USA            |11316 |
|Lagos, Nigeria |9989  |
|California, USA|8647  |
|Nigeria        |8615  |
|Washington, DC |7315  |
|Los Angeles, CA|7151  |
|Florida, USA   |7007  |
|Texas, USA     |6819  |
|Chicago, IL    |5711  |
|New York, NY   |4792  |
|New York, USA  |4551  |
|Houston, TX    |4016  |
|Atlanta, GA    |3929  |
|Earth          |3439  |
|Texas          |3420  |
|Canada         |3391  |
|India          |3175  |
|London, England|3137  |
+---------------+------+
only showing top 20 rows



In [7]:
pip install a-world-of-countries

Note: you may need to restart the kernel to use updated packages.


In [8]:
import awoc

my_world = awoc.AWOC()

countries = my_world.get_countries()

In [9]:
df = df.withColumn("user_location_clean", lower(df["user.location"]))
df = df.withColumn("user_location_clean", regexp_replace(df["user_location_clean"], "[^a-zA-Z0-9 ]", " "))
df.limit(5)

coordinates,created_at,favorite_count,favorited,filter_level,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,lang,place,quote_count,quoted_status,quoted_status_id,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,entities,extended_entities,extended_tweet,geo,place.country,place.full_name,place.name,place.place_type,user.id,user.name,user.screen_name,user.location,user.created_at,user.description,user.followers_count,user.friends_count,user.listed_count,user.favourites_count,user.statuses_count,user.url,user.verified,retweets_count,user_type,hashtags,user_location_clean
null,Mon May 09 14:04:...,0,false,low,1523665166109646850,null,null,null,false,en,null,0,null,null,null,0,0,RT,funeunice,"{null, Mon May 09...","<a href=""http://t...",RT @funeunice: It...,1652105058587,false,It doesn’t matter...,"{false, Sun Feb 2...","{[], null, [], []...",null,null,null,null,null,null,null,1497825045015633924,Nala,nalacarson95,null,Sun Feb 27 06:45:...,"Animal lover, Rap...",67,21,0,6065,8687,null,false,16,other,[],null
null,Mon May 09 14:04:...,0,false,low,1523665269373259782,null,null,null,false,en,null,0,null,null,null,0,0,RT,akaOMOLUABI,"{null, Mon May 09...","<a href=""http://t...",RT @akaOMOLUABI: ...,1652105083207,false,Meet me at the Un...,"{false, Tue Sep 2...","{[], null, [], []...",null,null,null,null,null,null,null,1176551054852464640,Desmond 😊,Segxy_Victor,Unilorin,Tue Sep 24 17:37:...,Entertainer | Dan...,279,538,0,2492,1098,null,false,92,other,[],unilorin
null,Mon May 09 14:04:...,0,false,low,1523665317821841408,null,null,null,false,en,null,0,null,null,null,0,0,RT,akaOMOLUABI,"{null, Mon May 09...","<a href=""http://t...",RT @akaOMOLUABI: ...,1652105094758,false,Meet me at the Un...,"{false, Sat Jun 0...","{[], null, [], []...",null,null,null,null,null,null,null,1137277249420431360,satipn,BelloIdowu17,"Lagos, Nigeria",Sat Jun 08 08:36:...,Comic..Humor..Asc...,45,165,0,480,835,null,false,93,other,[],lagos nigeria
null,Mon May 09 14:04:...,0,false,low,1523665328911568896,null,null,null,false,en,null,0,null,null,null,0,0,RT,WajahatAli,"{null, Mon May 09...","<a href=""https://...",RT @WajahatAli: T...,1652105097402,false,The wealthiest an...,"{false, Tue Oct 1...","{[], null, [], []...",null,null,null,null,null,null,null,3945305295,Elsworth Amasa,wfrolik,"Pittsburgh, PA",Tue Oct 13 09:47:...,Professional dogw...,315,653,7,160724,342887,http://www.oneide...,false,42,other,[],pittsburgh pa
null,Mon May 09 14:05:...,0,false,low,1523665345562943494,null,null,null,false,en,null,0,null,null,null,0,0,,null,null,"<a href=""https://...",Big Tech Info Sha...,1652105101372,true,Big Tech Info Sha...,"{false, Sun Oct 1...","{[], null, [], [{...",null,"{[0, 274], {[], n...",null,null,null,null,null,919380078072684546,J Fish,JackinSJ,Southern NJ,Sun Oct 15 01:51:...,I am an Independe...,332,326,0,10221,10239,null,false,null,other,[],southern nj


In [13]:
#df = df.withColumn("user_location_clean",
 #                  when((df['user_location_state']) != 'Other', "United States")
 #                  .otherwise(col("user_location_clean")))

In [14]:
df.filter(col('user_location_clean')=='United States')

coordinates,created_at,favorite_count,favorited,filter_level,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,lang,place,quote_count,quoted_status,quoted_status_id,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,entities,extended_entities,extended_tweet,geo,place.country,place.full_name,place.name,place.place_type,user.id,user.name,user.screen_name,user.location,user.created_at,user.description,user.followers_count,user.friends_count,user.listed_count,user.favourites_count,user.statuses_count,user.url,user.verified,retweets_count,user_type,user_location_clean,user_location_state
null,Wed Nov 16 00:04:...,0,false,low,1592669828372131840,aopaderf,1592669361244098561,848970475589914624,false,en,null,0,null,null,null,0,0,,TheCalvinCooli1 V...,null,"<a href=""https://...",@bestgtor88 @TheC...,1668557052115,true,@bestgtor88 @TheC...,"{false, Mon Apr 0...","{[], null, [], [{...",null,"{[29, 307], {[], ...",null,null,null,null,null,848970475589914624,What the freak is...,aopaderf,"Phoenix, AZ",Mon Apr 03 18:48:...,Pilot-instrument ...,955,1946,7,52966,28157,null,false,null,other,United States,AZ
null,Wed Nov 16 00:04:...,0,false,low,1592670023512121344,darinaras,1592635639379693568,32597170,false,en,null,0,null,null,null,0,0,,NoLieWithBTC If y...,null,"<a href=""http://t...",@darinaras @NoLie...,1668557098640,true,@darinaras @NoLie...,"{false, Fri Oct 2...","{[], null, [], [{...",null,"{[25, 197], {[], ...",null,null,null,null,null,1586070527177027584,red beard,Steelguy74,"Washington, USA",Fri Oct 28 19:01:...,steel worker/outd...,18,177,0,704,140,null,false,null,other,United States,WA
null,Wed Nov 16 00:05:...,0,false,low,1592670185076723713,joshuarhett,1592210731503849473,114773910,false,en,null,0,null,null,null,0,0,,AnnieBeeson @SenW...,null,"<a href=""https://...",@joshuarhett @Ann...,1668557137160,true,@joshuarhett @Ann...,"{false, Sun May 2...","{[], null, [], [{...",null,"{[49, 213], {[], ...",null,null,null,null,null,1530714373962051585,ANTIWOKE4261,CarlRupp1007,"Green Bay,WI",Sun May 29 00:55:...,"MENSA MEMBER, DIS...",5,16,0,27,266,null,false,null,other,United States,WI
null,Wed Nov 16 00:06:...,0,false,low,1592670362151817218,null,null,null,false,en,null,0,null,null,null,0,0,,null,null,"<a href=""https://...",Providence school...,1668557179378,true,Providence school...,"{false, Thu Feb 1...","{[], null, [], [{...",null,"{[0, 243], {[], n...",null,null,null,null,null,3017796337,Ray Jackson,ZiloElGrande,"Las Vegas, NV",Thu Feb 12 21:17:...,Vietnam Vet,5453,5834,7,364083,62658,null,false,null,other,United States,NV
null,Wed Nov 16 00:07:...,0,false,low,1592670642033553409,null,null,null,false,en,null,0,null,null,null,0,0,,null,null,"<a href=""https://...",🔴🎥𝐖𝐀𝐓𝐂𝐇 𝐋...,1668557246107,true,🔴🎥𝐖𝐀𝐓𝐂𝐇 𝐋...,"{false, Thu May 1...","{[], null, [], [{...",null,"{[0, 143], {[], n...",null,null,null,null,null,575810483,Pettymerlin,petty1112merlin,California,Thu May 10 01:00:...,null,97,273,0,20,9355,null,false,null,other,United States,CA
null,Wed Nov 16 00:07:...,0,false,low,1592670674262556672,null,null,null,false,en,null,0,null,null,null,0,0,RT,AP,"{null, Tue Nov 15...","<a href=""https://...",RT @AP: Arizona v...,1668557253791,false,Arizona voters ha...,"{false, Thu Feb 0...","{[], null, [], []...",null,null,null,null,null,null,null,480875170,VonVictor V. Rose...,VonRosenchild,"New York, NY",Thu Feb 02 01:05:...,@HubBucket Inc Fo...,13662,8014,2356,1105719,1106915,https://hubbucket...,false,102,other,United States,NY
null,Wed Nov 16 00:08:...,0,false,low,1592670828864638976,null,null,null,false,en,null,0,null,null,null,0,0,RT,MorePerfectUS,"{null, Mon Nov 14...","<a href=""https://...",RT @MorePerfectUS...,1668557290651,false,"BREAKING: 48,000 ...","{false, Sun Mar 3...","{[], null, [], []...",null,null,null,null,null,null,null,1317650684,Nick Grandstaff,NGrandstaff,"Ithaca, New York",Sun Mar 31 03:36:...,move fast. do 

In [15]:
df.groupBy('user_location_clean').agg(countDistinct('user.id').alias('num_accounts')).show()

+--------------------+------------+
| user_location_clean|num_accounts|
+--------------------+------------+
|       she they   19|           7|
|              online|          71|
|  reality   lv l a  |           1|
|    bengaluru  india|         384|
|       san juan city|           2|
|  deep ellum  dallas|           1|
|  bbc beach  grenada|           1|
|where uteruses ar...|           1|
| somewhere  outthere|           2|
|       they she   17|           3|
|   united states    |           9|
|     barrie  ontario|          21|
|disappointment is...|           2|
|   snake river plain|           1|
|bluest of blue st...|           1|
|johns hopkins uni...|           4|
|somewhere you re ...|           4|
|  minna  niger state|           3|
|           traveling|          13|
|        here and now|          18|
+--------------------+------------+
only showing top 20 rows



In [13]:
from pyspark.sql.functions import lower, when, array_contains
from pyspark.sql.types import StringType
from pyspark.sql.functions import broadcast

# Create a broadcast variable for the countries list
broadcast_countries = sc.broadcast(countries)

# Define a UDF to match the user location to a country
@udf(StringType())
def match_country_udf(location):
    for country in broadcast_countries.value:
        if (location is not None and 
            (country['Country Name'].lower() in location)): #or 
             #country['ISO2'].tolower in location or 
             #country['ISO3'].tolower in location)):
            return country['Country Name']
    return location

# Apply the UDF to the DataFrame
df = df.withColumn('user_country', match_country_udf(lower(col('user_location_clean'))))

In [14]:
countries[:2]

[{'Country Name': 'Afghanistan',
  'ISO2': 'AF',
  'ISO3': 'AFG',
  'TLD': 'af',
  'FIPS': 'AF',
  'ISO Numeric': '004',
  'GeoNameID': '1149361',
  'E164': '93',
  'Phone Code': '93',
  'Continent Name': 'Asia',
  'Continent Code': 'as',
  'Capital': 'Kabul',
  'Time Zone in Capital': 'Asia/Kabul',
  'Currency Name': 'Afghani',
  'Languages': 'Afghan Persian or Dari (official) 50%, Pashto (official) 35%, Turkic languages (primarily Uzbek and Turkmen) 11%, 30 minor languages (primarily Balochi and Pashai) 4%, much bilingualism, but Dari functions as the lingua franca',
  'Area KM2': '647500'},
 {'Country Name': 'Albania',
  'ISO2': 'AL',
  'ISO3': 'ALB',
  'TLD': 'al',
  'FIPS': 'AL',
  'ISO Numeric': '008',
  'GeoNameID': '783754',
  'E164': '355',
  'Phone Code': '355',
  'Continent Name': 'Europe',
  'Continent Code': 'eu',
  'Capital': 'Tirana',
  'Time Zone in Capital': 'Europe/Tirane',
  'Currency Name': 'Lek',
  'Languages': 'Albanian 98.8% (official - derived from Tosk dialect)

In [14]:
df.limit(5)

coordinates,created_at,favorite_count,favorited,filter_level,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,lang,place,quote_count,quoted_status,quoted_status_id,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,entities,extended_entities,extended_tweet,geo,place.country,place.full_name,place.name,place.place_type,user.id,user.name,user.screen_name,user.location,user.created_at,user.description,user.followers_count,user.friends_count,user.listed_count,user.favourites_count,user.statuses_count,user.url,user.verified,retweets_count,user_type,user_location_clean,user_country
null,Mon Aug 29 21:05:...,0,false,low,1564358569885147138,null,null,null,false,en,null,0,null,null,null,0,0,RT,esserlaw,"{null, Mon Aug 29...","<a href=""http://t...",RT @esserlaw: @ax...,1661807122094,false,"@axios ""Black bor...","{false, Sun Aug 2...","{[], null, [], []...",null,null,null,null,null,null,null,1165424719719325697,dw retired teacher,retired_in_tx,"Texas, USA",Sun Aug 25 00:45:...,"Retired educator,...",1648,2592,1,482947,142222,null,false,1,other,texas usa,texas usa
null,Mon Aug 29 21:05:...,0,false,low,1564358572728877060,null,null,null,false,en,null,0,null,null,null,0,0,RT,CharlieEatonPhD,"{null, Mon Aug 29...","<a href=""http://t...",RT @CharlieEatonP...,1661807122772,false,This claims risin...,"{false, Fri Jul 1...","{[], null, [], []...",null,null,null,null,null,null,null,336015206,Janet Lorin,JanetLorin,New York,Fri Jul 15 16:21:...,Higher ed finance...,11543,3779,493,1396,18852,https://www.bloom...,true,33,other,new york,new york
null,Mon Aug 29 21:05:...,0,false,low,1564358595646410752,null,null,null,false,en,null,0,null,null,null,0,0,RT,ErichinATL,"{null, Sun Aug 28...","<a href=""http://t...",RT @ErichinATL: T...,1661807128236,false,The people who do...,"{false, Tue May 2...","{[], null, [], []...",null,null,null,null,null,null,null,1529190288115175425,LightSpeed7,Speed7Light,null,Tue May 24 19:59:...,null,83,206,0,38139,27016,null,false,9562,other,null,null
null,Mon Aug 29 21:05:...,0,false,low,1564358600168030209,null,null,null,false,en,null,0,null,null,null,0,0,RT,ErichinATL,"{null, Sun Aug 28...","<a href=""http://t...",RT @ErichinATL: T...,1661807129314,false,The people who do...,"{false, Tue Jun 1...","{[], null, [], []...",null,null,null,null,null,null,null,1404675995051905024,Forrest Walker,Forrest50664245,null,Tue Jun 15 05:44:...,enlightenment enf...,8,68,1,1827,3469,null,false,9563,other,null,null
null,Mon Aug 29 21:05:...,0,false,low,1564358616215420930,null,null,null,false,en,null,0,null,null,null,0,0,RT,ErichinATL,"{null, Sun Aug 28...","<a href=""http://t...",RT @ErichinATL: T...,1661807133140,false,The people who do...,"{false, Tue Apr 1...","{[], null, [], []...",null,null,null,null,null,null,null,556009783,hk47Mik,hk47Mik,"Upstate New York,...",Tue Apr 17 14:24:...,null,44,311,0,24587,1679,null,false,9564,other,upstate new york ...,upstate new york ...


In [18]:
df.groupBy('user_country').agg(countDistinct('user.id').alias('num_accounts')).show()

+--------------------+------------+
|        user_country|num_accounts|
+--------------------+------------+
|                Chad|          26|
|            Paraguay|          45|
|              Russia|         183|
|            Anguilla|           4|
|               Yemen|          18|
|             Senegal|          39|
|              Sweden|         310|
|              Guyana|          16|
|              Jersey|        3209|
|         Philippines|        1045|
|            Djibouti|          12|
|               Tonga|          18|
|           Singapore|         400|
|            Malaysia|        1107|
|                Fiji|          73|
|              Turkey|         127|
|              Malawi|         142|
|                Iraq|          61|
|             Germany|        1172|
|Northern Mariana ...|           7|
+--------------------+------------+
only showing top 20 rows



In [ ]:
df.groupBy('user.location').agg(countDistinct('user.id').alias('num_accounts')).show()

+--------------------+------------+
|            location|num_accounts|
+--------------------+------------+
|                Utah|         184|
|                 WI |           3|
|    Mid-Atlantic USA|           3|
|           Camas, WA|           8|
|   St Petersburg, FL|         158|
| Bayern, Deutschland|          13|
|     fort worth, tx |           1|
|              online|          13|
|       Côte d'Ivoire|          18|
|     Gainesville, FL|         186|
|              Heaven|          95|
|    South Hampton NY|           1|
|Ocean Front Prope...|           1|
|Accraaaaaaaaaaaaa...|           1|
|          Dragonland|           1|
|Tortilla in the w...|           1|
|  Seattle Washington|           7|
|    Albany, new york|           1|
| Mumbai, Maharashtra|          19|
| Brisbane, Australia|          54|
+--------------------+------------+
only showing top 20 rows



In [10]:
#from pyspark.sql.functions import lower, regexp_replace


In [11]:
df.select('user_location_clean').show(10, truncate=False)

+--------------------------+
|user_location_clean       |
+--------------------------+
|farakwai                  |
|phoenix  az               |
|null                      |
|null                      |
|india                     |
|washington  usa           |
|null                      |
|green bay wi              |
|london shakespeare country|
|null                      |
+--------------------------+
only showing top 10 rows



In [10]:
df = df.withColumn("user_location_state",
                    when(lower(df["user_location_clean"]).rlike("\\b(alabama|al)\\b"), "AL")
                    .when(lower(df["user_location_clean"]).rlike("\\b(alaska|ak)\\b"), "AK")
                    .when(lower(df["user_location_clean"]).rlike("\\b(arizona|az)\\b"), "AZ")
                    .when(lower(df["user_location_clean"]).rlike("\\b(arkansas|ar)\\b"), "AR")
                    .when(lower(df["user_location_clean"]).rlike("\\b(california|ca)\\b"), "CA")
                    .when(lower(df["user_location_clean"]).rlike("\\b(colorado|co)\\b"), "CO")
                    .when(lower(df["user_location_clean"]).rlike("\\b(connecticut|ct)\\b"), "CT")
                    .when(lower(df["user_location_clean"]).rlike("\\b(delaware|de)\\b"), "DE")
                    .when(lower(df["user_location_clean"]).rlike("\\b(florida|fl)\\b"), "FL")
                    .when(lower(df["user_location_clean"]).rlike("\\b(georgia|ga)\\b"), "GA")
                    .when(lower(df["user_location_clean"]).rlike("\\b(hawaii|hi)\\b"), "HI")
                    .when(lower(df["user_location_clean"]).rlike("\\b(idaho|id)\\b"), "ID")
                    .when(lower(df["user_location_clean"]).rlike("\\b(illinois|il)\\b"), "IL")
                    .when(lower(df["user_location_clean"]).rlike("\\b(indiana|in)\\b"), "IN")
                    .when(lower(df["user_location_clean"]).rlike("\\b(iowa|ia)\\b"), "IA")
                    .when(lower(df["user_location_clean"]).rlike("\\b(kansas|ks)\\b"), "KS")
                    .when(lower(df["user_location_clean"]).rlike("\\b(kentucky|ky)\\b"), "KY")
                    .when(lower(df["user_location_clean"]).rlike("\\b(louisiana|la)\\b"), "LA")
                    .when(lower(df["user_location_clean"]).rlike("\\b(maine|me)\\b"), "ME")
                    .when(lower(df["user_location_clean"]).rlike("\\b(maryland|md)\\b"), "MD")
                    .when(lower(df["user_location_clean"]).rlike("\\b(massachusetts|ma)\\b"), "MA")
                    .when(lower(df["user_location_clean"]).rlike("\\b(michigan|mi)\\b"), "MI")
                    .when(lower(df["user_location_clean"]).rlike("\\b(minnesota|mn)\\b"), "MN")
                    .when(lower(df["user_location_clean"]).rlike("\\b(mississippi|ms)\\b"), "MS")
                    .when(lower(df["user_location_clean"]).rlike("\\b(missouri|mo)\\b"), "MO")
                    .when(lower(df["user_location_clean"]).rlike("\\b(montana|mt)\\b"), "MT")
                    .when(lower(df["user_location_clean"]).rlike("\\b(nebraska|ne)\\b"), "NE")
                    .when(lower(df["user_location_clean"]).rlike("\\b(nevada|nv)\\b"), "NV")
                    .when(lower(df["user_location_clean"]).rlike("\\b(new hampshire|nh)\\b"), "NH")
                    .when(lower(df["user_location_clean"]).rlike("\\b(new jersey|nj)\\b"), "NJ")
                    .when(lower(df["user_location_clean"]).rlike("\\b(new mexico|nm)\\b"), "NM")
                    .when(lower(df["user_location_clean"]).rlike("\\b(new york|ny|nyc|big apple)\\b"), "NY")
                    .when(lower(df["user_location_clean"]).rlike("\\b(north carolina|nc)\\b"), "NC")
                    .when(lower(df["user_location_clean"]).rlike("\\b(north dakota|nd)\\b"), "ND")
                    .when(lower(df["user_location_clean"]).rlike("\\b(ohio|oh)\\b"), "OH")
                    .when(lower(df["user_location_clean"]).rlike("\\b(oklahoma|ok)\\b"), "OK")
                    .when(lower(df["user_location_clean"]).rlike("\\b(oregon|or)\\b"), "OR")
                    .when(lower(df["user_location_clean"]).rlike("\\b(pennsylvania|pa)\\b"), "PA")
                    .when(lower(df["user_location_clean"]).rlike("\\b(rhode island|ri)\\b"), "RI")
                    .when(lower(df["user_location_clean"]).rlike("\\b(south carolina|sc)\\b"), "SC")
                    .when(lower(df["user_location_clean"]).rlike("\\b(south dakota|sd)\\b"), "SD")
                    .when(lower(df["user_location_clean"]).rlike("\\b(tennessee|tn)\\b"), "TN")
                    .when(lower(df["user_location_clean"]).rlike("\\b(texas|tx)\\b"), "TX")
                    .when(lower(df["user_location_clean"]).rlike("\\b(utah|ut)\\b"), "UT")
                    .when(lower(df["user_location_clean"]).rlike("\\b(vermont|vt)\\b"), "VT")
                    .when(lower(df["user_location_clean"]).rlike("\\b(virginia|va)\\b"), "VA")
                    .when(lower(df["user_location_clean"]).rlike("\\b(washington|wa)\\b"), "WA")
                    .when(lower(df["user_location_clean"]).rlike("\\b(west virginia|wv)\\b"), "WV")
                    .when(lower(df["user_location_clean"]).rlike("\\b(wisconsin|wi)\\b"), "WI")
                    .when(lower(df["user_location_clean"]).rlike("\\b(wyoming|wy)\\b"), "WY")
                   
                    #.when(lower(df["user_location_state"]).rlike("\\b(united states|usa|us|u s a|u s| \
                    #america| the states| us of a| land of the free)\\b"), "USA")
                    .otherwise('Other'))

In [11]:
df.limit(5)

coordinates,created_at,favorite_count,favorited,filter_level,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,lang,place,quote_count,quoted_status,quoted_status_id,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,entities,extended_entities,extended_tweet,geo,place.country,place.full_name,place.name,place.place_type,user.id,user.name,user.screen_name,user.location,user.created_at,user.description,user.followers_count,user.friends_count,user.listed_count,user.favourites_count,user.statuses_count,user.url,user.verified,retweets_count,user_type,hashtags,user_location_clean,user_location_state
null,Mon Sep 26 08:47:...,0,false,low,1574319825777737728,null,null,null,false,en,null,0,null,null,null,0,0,RT,ushiebuaras,"{null, Sun Sep 25...","<a href=""http://t...",RT @ushiebuaras: ...,1664182070570,false,Interested in a M...,"{false, Tue Dec 1...","{[], null, [], []...",null,null,null,null,null,null,null,943102958492897280,Dr Leo Eze,ezebit2,"Lagos, Nigeria",Tue Dec 19 12:57:...,"Bitcoin saver, Me...",1248,996,19,2008,2024,null,false,88,other,[],lagos nigeria,Other
null,Mon Sep 26 08:47:...,0,false,low,1574319839149281280,null,null,null,true,en,null,0,"{null, Sat Sep 24...",1573804912265281537,why does the univ...,0,0,RT,JoshuaPotash,"{null, Mon Sep 26...","<a href=""https://...",RT @JoshuaPotash:...,1664182073758,false,People say free c...,"{false, Thu Oct 1...","{[], null, [], []...",null,null,null,null,null,null,null,1184946666832564224,Leyley,grlfromlebanon,null,Thu Oct 17 21:40:...,null,23,318,0,1344,696,null,false,5848,other,[],null,Other
null,Mon Sep 26 08:48:...,0,false,low,1574319955541008384,zppINFO,1574319951736979456,713945335,false,en,null,0,null,null,null,0,0,,null,null,"<a href=""http://t...",post. The school ...,1664182101508,true,post. The school ...,"{false, Tue Jul 2...","{[], null, [], [{...",null,"{[0, 219], {[], n...",null,null,null,null,null,713945335,The Zimbabwe Peac...,zppINFO,Zimbabwe,Tue Jul 24 08:34:...,#WINNER | @CSR_I...,16109,1003,36,947,5485,http://www.zimpea...,false,null,other,[],zimbabwe,Other
null,Mon Sep 26 08:49:...,0,false,low,1574320286933176321,null,null,null,false,en,null,0,null,null,null,0,0,,null,null,"<a href=""http://w...",#JobAlert new Tea...,1664182180518,true,#JobAlert new Tea...,"{false, Thu Oct 0...","{[{[0, 9], JobAle...",null,"{[0, 206], {[{[0,...",null,null,null,null,null,784004583890776064,Northants Teaching,Northants_Teach,Northamptonshire,Thu Oct 06 12:17:...,Recruiting except...,1500,638,6,2024,10192,http://www.teachn...,false,null,other,[JobAlert],northamptonshire,Other
null,Mon Sep 26 08:49:...,0,false,low,1574320303726993409,null,null,null,true,en,null,0,"{null, Sat Sep 24...",1573804912265281537,why does the univ...,0,0,RT,JoshuaPotash,"{null, Mon Sep 26...","<a href=""https://...",RT @JoshuaPotash:...,1664182184522,false,People say free c...,"{false, Wed Nov 0...","{[], null, [], []...",null,null,null,null,null,null,null,796364661495136257,BoJess,bojess1984,"Edinburg, TX",Wed Nov 09 14:51:...,null,163,370,0,87267,46942,null,false,5857,other,[],edinburg tx,TX


In [11]:
df.select("user.location").distinct().count()

185256

In [26]:
df.select("user_location_state").distinct().count()

51

In [23]:
df_location_counts = df.groupBy("user_location_state").count().orderBy(desc("count"))
df_location_counts.show(20, truncate=False)

+-------------------+-------+
|user_location_state|count  |
+-------------------+-------+
|Other              |1085136|
|CA                 |52707  |
|TX                 |39687  |
|NY                 |35326  |
|FL                 |29463  |
|WA                 |21567  |
|IN                 |20982  |
|PA                 |15908  |
|IL                 |14795  |
|OH                 |13678  |
|NC                 |12592  |
|GA                 |12455  |
|AZ                 |12438  |
|VA                 |11655  |
|MI                 |11649  |
|MA                 |11326  |
|NJ                 |9691   |
|CO                 |9391   |
|OR                 |8618   |
|TN                 |7874   |
+-------------------+-------+
only showing top 20 rows



In [17]:
df_location_counts = df.groupBy("user_country").count().orderBy(desc("count"))
df_location_counts.show(30, truncate=False)

+----------------+------+
|user_country    |count |
+----------------+------+
|null            |585096|
|Niger           |33479 |
|United States   |32062 |
|India           |20803 |
|usa             |11680 |
|Canada          |10881 |
|california  usa |8662  |
|washington  dc  |7342  |
|los angeles  ca |7242  |
|florida  usa    |7018  |
|texas  usa      |6829  |
|United Kingdom  |6087  |
|Pakistan        |6035  |
|Ghana           |5915  |
|Jersey          |5832  |
|chicago  il     |5784  |
|Kenya           |5150  |
|new york  ny    |4801  |
|South Africa    |4593  |
|new york  usa   |4551  |
|houston  tx     |4103  |
|Georgia         |4071  |
|atlanta  ga     |3996  |
|earth           |3850  |
|texas           |3766  |
|Australia       |3756  |
|london  england |3139  |
|boston  ma      |3033  |
|Uganda          |2981  |
|philadelphia  pa|2979  |
+----------------+------+
only showing top 30 rows



In [16]:
df.select(countDistinct(col("user_country"))).collect()[0][0]

148503

23/03/09 17:55:22 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_13_python !


In [18]:
pip install plotly


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 60.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
df.createOrReplaceTempView("users")

In [30]:
user_by_state = spark.sql("""
SELECT user_location_state, count(*) as tweet_count
    FROM users
    GROUP BY user_location_state
""")

In [31]:
user_by_state.show()

+-------------------+-----------+
|user_location_state|tweet_count|
+-------------------+-----------+
|                 AZ|      12438|
|                 SC|       5422|
|                 LA|       5932|
|                 MN|       7228|
|                 NJ|       9691|
|                 OR|       8618|
|                 VA|      11655|
|                 RI|       1588|
|                 KY|       5083|
|                 WY|        561|
|                 NH|       2093|
|                 MI|      11649|
|                 NV|       4366|
|                 WI|       5767|
|                 ID|       1875|
|                 CA|      52707|
|                 NE|       2682|
|                 CT|       4369|
|                 MT|       1455|
|                 NC|      12592|
+-------------------+-----------+
only showing top 20 rows



In [33]:
from pyspark.sql.functions import year, month, date_format

tweets_per_state = df.filter(year("created_at") == 2022).filter(month("created_at") == 8) \
                    .groupBy("user_location_state", date_format("created_at", "MMMM yyyy").alias("month")) \
                    .count().orderBy("user_location_state", "month")

In [34]:
tweets_per_state.show()

+-------------------+-----------+-----+
|user_location_state|      month|count|
+-------------------+-----------+-----+
|                 AK|August 2022|  370|
|                 AL|August 2022| 1646|
|                 AR|August 2022| 1016|
|                 AZ|August 2022| 3855|
|                 CA|August 2022|16694|
|                 CO|August 2022| 3186|
|                 CT|August 2022| 1402|
|                 DE|August 2022|  755|
|                 FL|August 2022| 9828|
|                 GA|August 2022| 3825|
|                 HI|August 2022|  528|
|                 IA|August 2022| 1183|
|                 ID|August 2022|  520|
|                 IL|August 2022| 4555|
|                 IN|August 2022| 5864|
|                 KS|August 2022| 1526|
|                 KY|August 2022| 1595|
|                 LA|August 2022| 1839|
|                 MA|August 2022| 3380|
|                 MD|August 2022| 2361|
+-------------------+-----------+-----+
only showing top 20 rows



In [37]:
tweets_per_state.write.csv('gs://msca-bdp-students-bucket/shared_data/monicazhang/tweets_per_state_aug.csv', header=True)

In [16]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("example").config("spark.sql.legacy.timeParserPolicy", "LEGACY").getOrCreate()

In [17]:
from pyspark.sql.functions import to_timestamp, date_format

df = df.withColumn("created_at", to_timestamp(df["created_at"], "EEE MMM dd HH:mm:ss ZZZZ yyyy"))

# Group the tweets by day and count the number of tweets in each group
tweets_per_date = df.groupBy(date_format("created_at", "yyyy-MM").alias("date")).count()


In [18]:
tweets_per_date.show()

+-------+------+
|   date| count|
+-------+------+
|2022-04|198898|
|2022-06|119158|
|2022-08|427035|
|2022-07| 90008|
|2023-02| 20309|
|2022-11|116809|
|2022-10|107247|
|2023-01| 94890|
|2022-09|127979|
|2022-05|162272|
|2022-12| 77050|
+-------+------+



In [25]:
user_by_state = spark.sql("""
SELECT user_location_state, count(*)
    FROM users
    WHERE DATE_FORMAT(created_at, 'yyyy-MM') = '2022-08'
    GROUP BY user_location_state
""")

In [26]:
user_by_state.show()

+-------------------+--------+
|user_location_state|count(1)|
+-------------------+--------+
+-------------------+--------+



In [ ]:
user_by_country = spark.sql("""
SELECT user_country, count(*)
    FROM users
    GROUP BY user_country
""")

In [ ]:
import plotly.express as px

# Convert the Spark DataFrame to a Pandas DataFrame
user_by_state_pd = user_by_state.toPandas()

# Create the choropleth map
fig = px.choropleth(user_by_state_pd, 
                    locations='user_location_state', 
                    locationmode='USA-states', 
                    color='count(1)',
                    scope='usa',
                    hover_name='user_location_state',
                    title='Tweet Count by US State')
fig.show()

In [32]:
fig.show()

23/03/09 17:16:14 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_19_python !


In [31]:
import datetime
import pytz

datetime.datetime.now(pytz.timezone('US/Central')).strftime("%a, %d %B %Y %H:%M:%S")

'Thu, 09 March 2023 11:15:34'

In [19]:
pip install geopy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 6.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_app")

In [22]:
import folium, geopy

state_counts = (df.groupBy("user_location_state")
                  .agg(count("*").alias("count"))
                  .select("user_location_state", "count")
                  .limit(10000).toPandas())

# Create a Folium map centered on the United States
map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

for index, row in state_counts.iterrows():
    state = row['user_location_state']
    location = geolocator.geocode(state)
    if location:
        marker = folium.Marker(location=[location.latitude, location.longitude])
        marker.add_to(map)


In [24]:
map

In [31]:
df.select("user_location_state").distinct().count()

112590

In [36]:
state_counts = (df.groupBy("user_location_state")
                  .agg(count("*").alias("count"))
                  .select("user_location_state", "count")
                  .limit(100000).toPandas())

# Create a Folium map centered on the United States
map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Define a color scale for the markers
color_scale = folium.LinearColormap(['green', 'yellow', 'red'], vmin=state_counts['count'].min(), vmax=state_counts['count'].max())

for index, row in state_counts.iterrows():
    state = row['user_location_state']
    count = row['count']
    location = geolocator.geocode(state)
    if location:
        # Set the marker color based on the count of tweets
        color = color_scale(count)
        marker = folium.Marker(location=[location.latitude, location.longitude], 
                               tooltip=f"{state}: {count} tweets",
                               icon=folium.Icon(color=color))
        marker.add_to(map)

# Add the color scale to the map
color_scale.caption = 'Number of tweets'
color_scale.add_to(map)

# Save the map to an HTML file
map.save('tweets_map.html')


In [37]:
from pyspark.sql.functions import count
import folium
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="twitter_map")

state_counts = (df.groupBy("user_location_state")
                  .agg(count("*").alias("count"))
                  .select("user_location_state", "count")
                  .limit(100000).toPandas())

# Create a Folium map centered on the United States
map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

color_scale = folium.LinearColormap(['green', 'yellow', 'red'], vmin=state_counts['count'].min(), vmax=state_counts['count'].max())

for index, row in state_counts.iterrows():
    state = row['user_location_state']
    count = row['count']
    location = geolocator.geocode(state)
    if location:
        # Set the marker color based on the count of tweets
        color = color_scale(count)
        marker = folium.Marker(location=[location.latitude, location.longitude], 
                               tooltip=f"{state}: {count} tweets",
                               icon=folium.Icon(color=color))
        marker.add_to(map)

# Add the color scale to the map
color_scale.caption = 'Number of tweets'
color_scale.add_to(map)

# Save the map to an HTML file
map.save('tweets_map.html')


In [38]:
import datetime
import pytz

datetime.datetime.now(pytz.timezone('US/Central')).strftime("%a, %d %B %Y %H:%M:%S")

'Thu, 09 March 2023 00:11:21'

In [43]:
map

In [42]:
state_counts.to_csv('gs://msca-bdp-students-bucket/shared_data/monicazhang/state_counts.csv', index=False)